In [1]:
%%capture
!pip install qldpc

In [2]:
from sympy.abc import x, y

import qldpc

# Bivariate bicycle codes

Bivariate bicycle codes (`BBCode`s) were first introduced in [this work](https://www.nature.com/articles/s41586-024-07107-7) by IBM (also at [arXiv:2308.07915](https://arxiv.org/pdf/2308.07915v2)).

`BBCode`s are built from:
- two cyclic group orders
- two polynomials in the generators of those cyclic groups

In [3]:
# construct the second-to-last code in Table 3 of arXiv:2308.07915v2, with code parameters [n, k, d] = [360, 12, <=24]
orders = {x: 30, y: 6}
poly_a = x**9 + y + y**2
poly_b = y**3 + x**25 + x**26
code = qldpc.codes.BBCode(orders, poly_a, poly_b)

print(code)
print()
print("number of logical qubits:", code.dimension)

# find an upper bound to the code distance with 100 Monte Carlo trials
# 100 trials is likely not enough to reach the upper bound of 24 found by IBM
print("code distance: <=", code.get_distance_bound(num_trials=100))

BBCode on 360 qubits with cyclic group orders {x: 30, y: 6} and generating polynomials
  A = x**9 + y**2 + y
  B = x**26 + x**25 + y**3

number of logical qubits: 12
code distance: <= 30


## Toric qubit layouts

A toric layout of a bivariate bicycle code is one in which: when all qubits are placed on a torus (that is, a rectangular grid with periodic boundary conditions), every check qubit addresses its four nearest neighbors (in addition to some distant data qubits).  In this sense, a `BBCode` with a toric layout can be seen as a toric code whose parity checks are "augmented" with non-local connections.

A `BBCode` whose generating polynomials look like `A = 1 + x + ...` and `B = 1 + y + ...` have "manifestly toric" layouts, in the sense that the "canonical" layout of the `BBCode` is a toric layout.  See `help(qldpc.codes.BBCode)` for more information and a description of the "canonical" layout of a `BBCode`.

In [4]:
# identify equivalent versions of this code that have "manifestly toric" layouts
for orders, poly_a, poly_b in code.get_equivalent_toric_layout_code_data():
    print(qldpc.codes.BBCode(orders, poly_a, poly_b))

BBCode on 360 qubits with cyclic group orders {x: 30, y: 6} and generating polynomials
  A = x**5*y**3 + x + 1
  B = x**9/y + y + 1
BBCode on 360 qubits with cyclic group orders {x: 30, y: 6} and generating polynomials
  A = x + 1 + y**3/x**4
  B = y + 1 + 1/(x**9*y)
BBCode on 360 qubits with cyclic group orders {x: 30, y: 6} and generating polynomials
  A = x**5*y**3 + x + 1
  B = x**9*y**2 + y + 1
BBCode on 360 qubits with cyclic group orders {x: 30, y: 6} and generating polynomials
  A = x + 1 + y**3/x**4
  B = y + 1 + y**2/x**9


## Qubit layouts that minimize communication distance

Find qubit layouts of `BBCode`s with minimal real-space qubit communication distances.  That is, these layouts try to minimize the maximum distance $D_\mathrm{max}$ between any check qubit and the data qubits that it must address, when all qubits are laid out on a rectangular grid.

The qubit placement strategy is described in [arXiv:2404.18809](https://arxiv.org/pdf/2404.18809v2).

In [5]:
# import some methods from the file find_bbcode_layouts.py, which should be in the same directory as this notebook
from find_bbcode_layouts import (
    find_layout_params,
    get_best_known_layout_params,
    get_qubit_pos_func,
    get_max_comm_distance,
)

# consider "folded" qubit layouts, as in Figure 2 of arXiv:2404.18809
folded_layout = True

# loop over all codes in Table 1 of arXiv:2404.18809
for code in [
    qldpc.codes.BBCode(
        {x: 6, y: 6},
        x**3 + y + y**2,
        y**3 + x + x**2,
    ),
    qldpc.codes.BBCode(
        {x: 15, y: 3},
        x**9 + y + y**2,
        1 + x**2 + x**7,
    ),
    qldpc.codes.BBCode(
        {x: 9, y: 6},
        x**3 + y + y**2,
        y**3 + x + x**2,
    ),
    qldpc.codes.BBCode(
        {x: 12, y: 6},
        x**3 + y + y**2,
        y**3 + x + x**2,
    ),
    qldpc.codes.BBCode(
        {x: 12, y: 12},
        x**3 + y**2 + y**7,
        y**3 + x + x**2,
    ),
]:
    print()
    print(code)
    print("number of logical qubits:", code.dimension)

    layout_params = get_best_known_layout_params(code, folded_layout)
    # layout_params = find_layout_params(code, folded_layout, verbose=False)

    # print qubit positions
    get_qubit_pos = get_qubit_pos_func(code, layout_params)
    # for node in sorted(code.graph.nodes):
    #     print(node, get_qubit_pos(node))

    max_distance = get_max_comm_distance(code, layout_params)
    print("maximum communication distance:", max_distance)


BBCode on 72 qubits with cyclic group orders {x: 6, y: 6} and generating polynomials
  A = x**3 + y**2 + y
  B = x**2 + x + y**3
number of logical qubits: 12
maximum communication distance: 5.0

BBCode on 90 qubits with cyclic group orders {x: 15, y: 3} and generating polynomials
  A = x**9 + y**2 + y
  B = x**7 + x**2 + 1
number of logical qubits: 8
maximum communication distance: 10.0

BBCode on 108 qubits with cyclic group orders {x: 9, y: 6} and generating polynomials
  A = x**3 + y**2 + y
  B = x**2 + x + y**3
number of logical qubits: 8
maximum communication distance: 7.0

BBCode on 144 qubits with cyclic group orders {x: 12, y: 6} and generating polynomials
  A = x**3 + y**2 + y
  B = x**2 + x + y**3
number of logical qubits: 12
maximum communication distance: 7.211102550927978

BBCode on 288 qubits with cyclic group orders {x: 12, y: 12} and generating polynomials
  A = x**3 + y**7 + y**2
  B = x**2 + x + y**3
number of logical qubits: 12
maximum communication distance: 7.2111